## Import and draw samples

In [ ]:
from scipy.stats import norm
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import logsumexp



def draw_from_normal_distr(shape, mu, var):
  rv = scipy.stats.norm(loc = mu, scale = np.sqrt(var))
  return rv.rvs(shape)


true_mu, true_q, true_v = np.array([0,1]), np.array([[0.95,0.05],[0.3,0.7]]), 0.5


def simulate_trajectory(size = 500, mu = true_mu, q = true_q, v = true_v):
  trajectory_states = []
  trajectory_observations = []

  noise =  draw_from_normal_distr(size, 0, v)

  trajectory_states.append(np.random.randint(len(mu)))
  trajectory_observations.append(trajectory_states[-1] + noise[0])

  old_state = trajectory_states[-1]

  for k in range(size-1):
    new_state = np.sum(np.cumsum(q[old_state,:]) <= np.random.rand())
    old_state = new_state

    trajectory_states.append(new_state)
    trajectory_observations.append(new_state + noise[k+1])

  return np.array(trajectory_states),np.array(trajectory_observations)


states, observations = simulate_trajectory(size = 8000)



init_mu, init_q, init_v = np.array([-0.5,0.5]), np.array([[0.7,0.3], [0.5,0.5]]), 2


## Batch EM and test

In [ ]:
from scipy.special import logsumexp


class batchEM():
  def __init__(self,observations, mu, q, v, eps = 1e-6):
    '''
    observations: the observations Y
    mu : mu0 size m, where m = number of states
    q: q0 (m,m)
    v: v0 noise
    '''

    self.observations = observations
    self.mu = mu
    self.q = np.log(q)
    self.v = v
    self.m = len(mu)

    self.pdf = self.compute_pdf()
    self.eps = eps

  def compute_pdf(self):
    pdf = self.observations[:,np.newaxis] - self.mu[np.newaxis, :]
    pdf = -pdf*pdf/(2*self.v) - np.log(2*np.pi*self.v)/2
    return pdf

  def compute_forward(self):
    '''
    '''
    forward = self.q[np.newaxis,:,:] + self.pdf[:,:,np.newaxis]
    forward[0,:,:] = forward[0,:,:] - np.log(self.m)

    for t in range(1,len(self.observations)-1):
      forward[t,:,:] = forward[t,:,:]+ logsumexp(forward[t-1,:,:], axis = 0, keepdims = True).T

    return forward[:-1,:,:]

  def compute_backward(self):
    backward = self.pdf.copy()

    for t in range(0,len(self.observations)-1, -1):
      backward[t,:] = backward[t,:]+ logsumexp(self.q + backward[t+1,np.newaxis,:], axis = 1)
    
    return backward

  def step(self):
    #E-Step
    self.pdf = self.compute_pdf()
    forward = self.compute_forward()
    backward = self.compute_backward()

    #ATTENTION
    self.S_q = (forward + backward[1:, np.newaxis,:])
    self.S_q = logsumexp(self.S_q - logsumexp(self.S_q, axis = (1,2), keepdims = True), axis = 0, b= 1/len(self.S_q))

    proba = backward.copy()
    proba[1:,:] = proba[1:,:]+logsumexp(forward,axis = 1)
    proba = proba - logsumexp(proba, axis = 1,keepdims = True)

    self.S_g0 = np.mean( np.exp(proba), axis = 0)
    self.S_g1 = np.mean(np.exp(proba)*self.observations[:,np.newaxis], axis = 0)
    self.S_g2 = np.mean(np.exp(proba)*np.square(self.observations[:,np.newaxis]), axis = 0)


    #M-Step
    self.q = self.S_q - logsumexp(self.S_q, axis = 1, keepdims = True)
    self.mu = self.S_g1/self.S_g0
#    self.v = (np.mean(np.square(self.observations)) - np.exp(logsumexp(2*self.S_g1-self.S_g0)))/np.exp(logsumexp(self.S_g0))
    self.v = np.sum(self.S_g2 - np.square(self.mu)*self.S_g0)/np.sum(self.S_g0)


    #self.v = (np.mean(np.square(self.observations)) - np.sum(self.mu*self.mu*np.exp(self.S_g0)))/np.exp(logsumexp(self.S_g0))

  def give_parameters(self):
    return np.exp(self.q), self.mu, self.v



In [ ]:
np.seterr(divide='raise', invalid='raise')

test = batchEM(observations, init_mu, init_q, init_v)

for k in range(100):
  test.step()
  if k % 10 == 0: 
    #print(test.give_parameters())
    pass

test.give_parameters()

In [ ]:
iter = 8000
batchEM_8000_q11 = []
batchEM_8000_q22 = []
batchEM_8000_mu1 = []
batchEM_8000_mu2 = []

N_TOT = 100

In [ ]:
for j in range(N_TOT):
  states, observations = simulate_trajectory(size = iter)
  EM = batchEM(observations, init_mu, init_q, init_v)
  q11 = []
  q22 = []
  mu1 = []
  mu2 = []

  for k in range(100):
    EM.step()
    q_pred, mu_pred, v_pred = EM.give_parameters()
    max_idx = np.argmax(mu_pred)
    min_idx = np.argmin(mu_pred)
    mu1.append(np.max(mu_pred))
    mu2.append(np.min(mu_pred))
    q11.append(q_pred[max_idx, max_idx])
    q22.append(q_pred[min_idx, min_idx])

  batchEM_8000_q11.append(q11)
  batchEM_8000_q22.append(q22)
  batchEM_8000_mu1.append(mu1)
  batchEM_8000_mu2.append(mu2)


In [ ]:
iter = 500
batchEM_500_q11 = []
batchEM_500_q22 = []
batchEM_500_mu1 = []
batchEM_500_mu2 = []

N_TOT = 100

In [ ]:
for j in range(N_TOT):
  states, observations = simulate_trajectory(size = iter)
  EM = batchEM(observations, init_mu, init_q, init_v)
  q11 = []
  q22 = []
  mu1 = []
  mu2 = []

  for k in range(100):
    EM.step()
    q_pred, mu_pred, v_pred = EM.give_parameters()
    max_idx = np.argmax(mu_pred)
    min_idx = np.argmin(mu_pred)
    mu1.append(np.max(mu_pred))
    mu2.append(np.min(mu_pred))
    q11.append(q_pred[max_idx, max_idx])
    q22.append(q_pred[min_idx, min_idx])

  batchEM_500_q11.append(q11)
  batchEM_500_q22.append(q22)
  batchEM_500_mu1.append(mu1)
  batchEM_500_mu2.append(mu2)


In [ ]:
batchEM_500_q11 = np.array(batchEM_500_q11)
batchEM_500_q22 = np.array(batchEM_500_q22)
batchEM_500_mu1 = np.array(batchEM_500_mu1)
batchEM_500_mu2 = np.array(batchEM_500_mu2)

In [ ]:
plt.style.use('Solarize_Light2')

batchEM_8000_q11 = np.array(batchEM_8000_q11)
batchEM_8000_q22 = np.array(batchEM_8000_q22)
batchEM_8000_mu1 = np.array(batchEM_8000_mu1)
batchEM_8000_mu2 = np.array(batchEM_8000_mu2)

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)

plt.plot(np.median(batchEM_8000_q11, axis = 0), 'g', label = 'batch size = 8000')
plt.plot(np.quantile(batchEM_8000_q11, .25, axis = 0), '--g')
plt.plot(np.quantile(batchEM_8000_q11, .75, axis = 0),  '--g')

plt.plot(np.median(batchEM_500_q11, axis = 0), 'b', label = 'batch size = 500')
plt.plot(np.quantile(batchEM_500_q11, .25, axis = 0), '--b')
plt.plot(np.quantile(batchEM_500_q11, .75, axis = 0),  '--b')

plt.hlines(true_q[1,1], 0, 100, 'r', linestyles = 'dotted', label = 'true q22')
plt.legend()
plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = (10,5)

plt.plot(np.median(batchEM_8000_mu1, axis = 0), 'g', label = 'batch size = 8000')
plt.plot(np.quantile(batchEM_8000_mu1, .25, axis = 0), '--g')
plt.plot(np.quantile(batchEM_8000_mu1, .75, axis = 0),  '--g')

plt.plot(np.median(batchEM_500_mu1, axis = 0), 'b', label = 'batch size = 500')
plt.plot(np.quantile(batchEM_500_mu1, .25, axis = 0), '--b')
plt.plot(np.quantile(batchEM_500_mu1, .75, axis = 0),  '--b')

plt.hlines(true_mu[1], 0, 100, 'r', linestyles = 'dotted', label = 'true mu2')
plt.legend()
plt.show()


## Online EM

In [ ]:


class onlineEM():
  def __init__(self,first_observation, mu, q, v, eps = 1e-6):
    '''
    observations: the observations Y
    mu : mu0 size m, where m = number of states
    q: q0 (m,m)
    v: v0 noise
    '''
    self.eps = eps
    self.mu = mu
    self.q = q
    self.v = v
    self.m = len(mu)

    pdf = self.compute_pdf_one_observation(first_observation)

    #init phi, rho_q, rho_g
    self.phi = pdf/np.sum(pdf)
    self.rho_q = np.zeros([self.m]*3)
    self.rho_g0 = np.identity(self.m)
    self.rho_g1 = first_observation*np.identity(self.m)
    self.rho_g2 = (first_observation**2)*np.identity(self.m)

  def compute_pdf_one_observation(self, one_observation):
    '''
    returns an vector of size m (log_pdf)
    '''
    pdf = one_observation - self.mu
    pdf = -(pdf**2)/(2*self.v) # - np.log(2*np.pi*self.v)/2
    return np.exp(pdf)

  def compute_pdf(self):
    pdf = self.observations[:,np.newaxis] - self.mu[np.newaxis, :]
    pdf = -(pdf*pdf)/(2*self.v) - np.log(2*np.pi*self.v)/2
    return pdf


  def step(self, observation, n, alpha = 0.6, burn = 8000):
    #Approximation FIlter Update

    #self.phi = self.phi@self.q*self.compute_pdf_one_observation(observation)
    #self.phi = self.phi/np.sum(self.phi)

    #Stochastic approximation E-step
    r = self.phi[:,np.newaxis]*self.q
    r = r/np.sum(r, axis = 0, keepdims = True)


    if np.any(self.compute_pdf_one_observation(observation) > 1):
      print("BEWARE, proba >0")
      print(observation, self.mu, np.exp(self.compute_pdf_one_observation(observation)))

    #print(np.exp(self.compute_pdf_one_observation(observation)))

    self.phi = np.sum(self.phi[:,np.newaxis]*self.q*self.compute_pdf_one_observation(observation)[np.newaxis, :], axis = 0) 
    self.phi = self.phi/np.sum(self.phi)



    if n <= burn:
      gamma = 0.01
    else:
      gamma = 1/(n+2 - burn)**alpha

    essai = self.rho_q@r*(1-gamma) + gamma*np.identity(self.m)[np.newaxis, :,:]*(r[:,:,np.newaxis])
    self.rho_q =  essai

    self.rho_g0 =  self.rho_g0@r*(1-gamma) + gamma*np.identity(self.m)
    self.rho_g1 =  self.rho_g1@r*(1-gamma) + gamma*np.identity(self.m)*observation
    self.rho_g2 =  self.rho_g2@r*(1-gamma) + gamma*np.identity(self.m)*(observation**2)


    #M-step
    S_q = self.rho_q@self.phi

    S_g0 = self.rho_g0@self.phi
    S_g1 = self.rho_g1@self.phi
    S_g2 = self.rho_g2@self.phi
    #print(self.rho_g1, np.exp(self.phi), S_g1)
    #print(S_g1, S_g0)


    self.q = S_q/np.sum(S_q, axis = 1, keepdims = True)
    self.mu = S_g1/S_g0
    self.v = np.sum(S_g2 - np.square(self.mu)*S_g0)/np.sum(S_g0)


  def give_parameters(self):
    return (self.q), self.mu, self.v



In [ ]:
np.seterr(divide='ignore', invalid='raise')

def simulate_observation_from_previous_state(state, mu = true_mu, q = true_q, v = true_v):

  noise =  draw_from_normal_distr(1, 0, v)
  new_state = np.sum(np.cumsum(q[state,:]) <= np.random.rand())


  return new_state,new_state + noise[0]


states, observations = simulate_trajectory(size = 1)

current_state = states[-1]
test = onlineEM(observations[-1], init_mu, init_q, init_v)


In [ ]:

np.seterr(divide='ignore', invalid='raise')
burn = 0


for k in range(10000):
  if k %1000 == 0:
    print(test.give_parameters())
  current_state, observation = simulate_observation_from_previous_state(current_state)

  test.step(observation,k, burn = burn)
print(test.give_parameters())



In [ ]:
import pandas as pd

df_online = pd.DataFrame(columns = ['mu1', 'mu2', 'q11', 'q22', 'nb_iterations', 'v'])
j = 0

In [ ]:
from datetime import datetime 
from IPython.display import clear_output


stopPoints = [500, 2000, 4000, 8000, 16000]


while j<100:
  clear_output(wait=True)
  print(j)

  states, observations = simulate_trajectory(size = 1)

  current_state = states[-1]
  EM = onlineEM(observations[-1], init_mu, init_q, init_v)
  time = 0

  for k in range(16001):
    current_state, observation = simulate_observation_from_previous_state(current_state)

    start_time = datetime.now() 
    EM.step(observation,k, burn = burn)
    time_elapsed = (datetime.now() - start_time).total_seconds() 
    time += time_elapsed


    if k in stopPoints:
      q_pred, mu_pred, v_pred = EM.give_parameters()

      max_idx = np.argmax(mu_pred)
      min_idx = np.argmin(mu_pred)


      df_online = df_online.append({'mu1': np.min(mu_pred), 'mu2': np.max(mu_pred), 'q11': q_pred[min_idx, min_idx],
                                    'q22': q_pred[max_idx, max_idx], 'nb_iterations': k,
                                    'v': v_pred}, ignore_index=True)
      
  print(time/16000)
  j +=1


In [ ]:
df_online.boxplot(by ='nb_iterations', column =['q22'], grid = False)
plt.axhline(true_q[1,1], c='r')
plt.show()
print()

df_online.boxplot(by ='nb_iterations', column =['q11'], grid = False)
plt.axhline(true_q[0,0], c='r')
plt.show()
print()

df_online.boxplot(by ='nb_iterations', column =['mu1'], grid = False)
plt.axhline(true_mu[0], c='r')
plt.show()
print()

df_online.boxplot(by ='nb_iterations', column =['mu2'], grid = False)
plt.axhline(true_mu[1], c='r')
plt.show()

print()
df_online.boxplot(by ='nb_iterations', column =['v'], grid = False)
plt.axhline(true_v, c='r')
plt.show()

boxplot batch

In [ ]:
import pandas as pd

df_batch = pd.DataFrame(columns = ['mu1', 'mu2', 'q11', 'q22', 'nb_iterations', 'v'])

j = 0


In [ ]:
stopPoints = [500, 2000, 4000, 8000, 16000]

while j<100:
  clear_output(wait=True)
  print(j)


  for iter in stopPoints:
    states, observations = simulate_trajectory(size = iter)
    EM = batchEM(observations, init_mu, init_q, init_v)
    start_time = datetime.now() 
    for k in range(50):
      EM.step()
    time_elapsed = (datetime.now() - start_time).total_seconds() 
    print(time_elapsed)


    q_pred, mu_pred, v_pred = EM.give_parameters()

    max_idx = np.argmax(mu_pred)
    min_idx = np.argmin(mu_pred)


    df_batch = df_batch.append({'mu1': np.min(mu_pred), 'mu2': np.max(mu_pred), 'q11': q_pred[min_idx, min_idx],
                                  'q22': q_pred[max_idx, max_idx], 'nb_iterations': iter,
                                  'v': v_pred}, ignore_index=True)
      

  j +=1


In [ ]:
df_batch.boxplot(by ='nb_iterations', column =['q22'], grid = False)
plt.axhline(true_q[1,1], c='r')
plt.show()
print()

df_batch.boxplot(by ='nb_iterations', column =['q11'], grid = False)
plt.axhline(true_q[0,0], c='r')
plt.show()
print()

df_batch.boxplot(by ='nb_iterations', column =['mu1'], grid = False)
plt.axhline(true_mu[0], c='r')
plt.show()
print()

df_batch.boxplot(by ='nb_iterations', column =['mu2'], grid = False)
plt.axhline(true_mu[1], c='r')
plt.show()

print()
df_batch.boxplot(by ='nb_iterations', column =['v'], grid = False)
plt.axhline(true_v, c='r')
plt.show()

In [ ]:
from datetime import datetime 
from IPython.display import clear_output


stopPoints = [500, 2000, 4000, 8000, 16000]


while j<100:
  clear_output(wait=True)
  print(j)

  states, observations = simulate_trajectory(size = 1)
  try:

    current_state = states[-1]
    EM = onlineEM(observations[-1], init_mu, init_q, init_v)
    time = 0

    for k in range(16001):
      current_state, observation = simulate_observation_from_previous_state(current_state)

      start_time = datetime.now() 
      EM.step(observation,k, burn = burn)
      time_elapsed = (datetime.now() - start_time).total_seconds() 
      time += time_elapsed


      if k in stopPoints:
        q_pred, mu_pred, v_pred = EM.give_parameters()

        max_idx = np.argmax(mu_pred)
        min_idx = np.argmin(mu_pred)


        df_online = df_online.append({'mu1': np.min(mu_pred), 'mu2': np.max(mu_pred), 'q11': q_pred[min_idx, min_idx],
                                      'q22': q_pred[max_idx, max_idx], 'nb_iterations': k,
                                      'v': v_pred}, ignore_index=True)
    except:
      pass

  print(time/16000)
  j +=1


## Draft



In [ ]:


class onlineEM():
  def __init__(self,first_observation, mu, q, v, eps = 1e-6):
    '''
    observations: the observations Y
    mu : mu0 size m, where m = number of states
    q: q0 (m,m)
    v: v0 noise
    '''
    self.eps = eps
    self.mu = mu
    self.q = np.log(q)
    self.v = v
    self.m = len(mu)

    pdf = self.compute_pdf_one_observation(first_observation)

    #init phi, rho_q, rho_g
    self.phi = pdf - logsumexp(pdf)#- np.log(self.m)
    self.rho_q = np.log(np.zeros([self.m]*3))
    self.rho_g0 = np.identity(self.m)
    self.rho_g1 = first_observation*np.identity(self.m)
    self.rho_g2 = (first_observation**2)*np.identity(self.m)

  def compute_pdf_one_observation(self, one_observation):
    '''
    returns an vector of size m (log_pdf)
    '''
    pdf = one_observation - self.mu
    pdf = -(pdf**2)/(2*self.v) # - np.log(2*np.pi*self.v)/2
    return pdf

  def compute_pdf(self):
    pdf = self.observations[:,np.newaxis] - self.mu[np.newaxis, :]
    pdf = -(pdf*pdf)/(2*self.v) - np.log(2*np.pi*self.v)/2
    return pdf


  def step(self, observation, n, alpha = 0.6, burn = 8000):
    #Approximation FIlter Update

    #self.phi = self.phi@self.q*self.compute_pdf_one_observation(observation)
    #self.phi = self.phi/np.sum(self.phi)

    #Stochastic approximation E-step
    r = self.phi[:,np.newaxis]+self.q
    r = r - logsumexp(r, axis = 0, keepdims = True)


    if np.any(self.compute_pdf_one_observation(observation) > 0):
      print("BEWARE, proba >0")
      print(observation, self.mu, np.exp(self.compute_pdf_one_observation(observation)))

    #print(np.exp(self.compute_pdf_one_observation(observation)))

    self.phi = logsumexp(self.phi[:,np.newaxis]+self.q+ self.compute_pdf_one_observation(observation)[np.newaxis, :], axis = 0) 
    self.phi = self.phi - logsumexp(self.phi)



    if n <= burn:
      gamma = 0.01
    else:
      gamma = 1/(n+2 - burn)**alpha

    #essai = np.exp(self.rho_q)@np.exp(r)*(1-gamma) + gamma*np.identity(self.m)[np.newaxis, :,:]*(np.exp(r)[:,:,np.newaxis])
    self.rho_q =  logsumexp(self.rho_q[:,:,:,np.newaxis] + r[np.newaxis, np.newaxis,:,:], axis = 2) + np.log(1-gamma)
    self.rho_q = np.logaddexp(self.rho_q,  r[:,:,np.newaxis] + np.log(gamma), where = np.identity(self.m)[np.newaxis, :,:] ==1)

    self.rho_g0 =  self.rho_g0@np.exp(r)*(1-gamma) + gamma*np.identity(self.m)
    self.rho_g1 =  self.rho_g1@np.exp(r)*(1-gamma) + gamma*np.identity(self.m)*observation
    self.rho_g2 =  self.rho_g2@np.exp(r)*(1-gamma) + gamma*np.identity(self.m)*(observation**2)


    #M-step
    S_q = logsumexp( self.rho_q + self.phi[np.newaxis, np.newaxis, :],axis = 2)

    S_g0 = self.rho_g0@np.exp(self.phi)
    S_g1 = self.rho_g1@np.exp(self.phi)
    S_g2 = self.rho_g2@np.exp(self.phi)
    #print(self.rho_g1, np.exp(self.phi), S_g1)
    #print(S_g1, S_g0)


    self.q = S_q - logsumexp(S_q, axis = 1, keepdims = True)
    self.mu = S_g1/S_g0
    self.v = np.sum(S_g2 - np.square(self.mu)*S_g0)/np.sum(S_g0)


  def give_parameters(self):
    return np.exp(self.q), self.mu, self.v



In [ ]:
def divide(a,b):
   return np.divide(a, b, out=np.zeros_like(a), where=b!=0)

class batchEM():
  def __init__(self,observations, mu, q, v, eps = 1e-6):
    '''
    observations: the observations Y
    mu : mu0 size m, where m = number of states
    q: q0 (m,m)
    v: v0 noise
    '''

    self.observations = observations
    self.mu = mu
    self.q = q
    self.v = v
    self.m = len(mu)

    self.pdf = self.compute_pdf()
    self.eps = eps

  def compute_pdf(self):
    pdf = self.observations[:,np.newaxis] - self.mu[np.newaxis, :]
    pdf = np.exp(-pdf*pdf/(2*self.v))/(np.sqrt(2*np.pi*self.v))
    return pdf

  def compute_forward(self):
    '''

    '''
    forward = self.q[np.newaxis,:,:]*self.pdf[:,:,np.newaxis]
    forward[0,:,:] = forward[0,:,:]/len(self.mu)

    for t in range(1,len(self.observations)-1):
      forward[t,:,:] = forward[t,:,:]*(np.sum(forward[t-1,:,:], axis = 0, keepdims = True).T)

    return forward[:-1,:,:]

  def compute_backward(self):
    backward = self.pdf.copy()

    for t in range(0,len(self.observations)-1, -1):
      backward[t,:] = backward[t,:]*(self.q@backward[t+1,:])
    
    return backward


  def step(self):
    #E-Step
    self.pdf = self.compute_pdf()
    forward = self.compute_forward()
    backward = self.compute_backward()

    #ATTENTION
    self.S_q = (forward*backward[1:, np.newaxis,:])
    self.S_q = np.mean(divide(self.S_q,np.sum(self.S_q, axis = (1,2), keepdims = True)),axis = 0)

    proba = backward.copy()
    proba[1:,:] = proba[1:,:]*np.sum(forward,axis = 1)
    proba = divide(proba,np.sum(proba, axis = 1,keepdims = True))
     
    self.S_g0 = np.mean( proba, axis = 0)
    self.S_g1 = np.mean(proba*self.observations[:,np.newaxis], axis = 0)


    #M-Step
    self.q = self.S_q/np.sum(self.S_q, axis = 1, keepdims = True)
    self.mu = self.S_g1/self.S_g0
    self.v = (np.mean(np.square(self.observations)) - np.sum(self.mu*self.mu*self.S_g0))/np.sum(self.S_g0)


  def give_parameters(self):
    return self.q, self.mu, self.v

